<a href="https://colab.research.google.com/github/sipocz/ingatlan/blob/master/ingatlan_com.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import requests
from bs4 import  BeautifulSoup
from google.colab import drive
import time
drive.mount('/content/drive',force_remount=True)



def formatphonenumber(stri):
    if stri[0]=="0":
        stri="3"+stri[1:]
    if len(stri)<10:
        stri="36"+stri    
    out="+"+stri[0:2]+"-("+stri[2:4]+")-"+stri[4:7]+"-"+stri[7:]
    
    return out

def clearphonenumber(stri):
    out=""
    
    for i in stri:
        if i.isnumeric():
            out=out+i
    return(formatphonenumber(out))


def clearengunit(stri):
    out=""
    
    for i in stri:
        if i in ["0","1","2","3","4","5","6","7","8","9"]:
            out=out+i
    return(out)

def clearengunit(stri):
    out=""
    
    for i in stri:
        if i in ["0","1","2","3","4","5","6","7","8","9"]:
            out=out+i
    return(out)

def onlynumber(stri):
    out=""
    
    for i in stri:
        if i in [",","0","1","2","3","4","5","6","7","8","9"]:
            out=out+i
    return(out)

def ansicode(sti):
    outstr=""
    for i in sti:
        a=i
        if i=="á":
            a="a"
        if i=="Á":
            a="A"
        if i=="é":
            a="e"
        if i=="É":
            a="E"
        if i=="í":
            a="I"
        if i=="Í":
            a="I"
        if i=="ö":
            a="o"
        if i=="Ö":
            a="O"
        if i=="ő":
            a="o"
        if i=="Ő":
            a="O"
        if i=="ó":
            a="o"
        if i=="Ó":
            a="O"
        if i=="ü":
            a="u"
        if i=="Ü":
            a="u"
        if i=="ú":
            a="u"
        if i=="Ú":
            a="u"
        outstr=outstr+a
    return(outstr)

   
        


def queryallinfo(url):
    header={'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36"}
    res=requests.get(url,headers=header)
    #print (res.text)
    soup=BeautifulSoup(res.text,"html.parser")
    print(soup)
    
   
    
    
    
    agentlist=soup.find_all("div", class_="parameter parameter-area-size")
    
    
   
    hazinfo=soup.find_all("div", title="parameter parameter-area-size")
    print(hazinfo)
    '''
    if len(hazinfo)>0:
        hazinfo=clearengunit(hazinfo[0].div.strong.text)
    else:
        hazterulet=0
    telekterulet= soup.find_all("li", title="Telekterület")
    #print(terulet)
    if len(telekterulet)>0:
        telekterulet=clearengunit(telekterulet[0].div.strong.text)
    else:
        telekterulet=0
    out=(agent,clearphonenumber(phone),hazinfo,telekterulet)
    return(out)
'''



def querycity(city,pmin=20,pmax=40):
    header={'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36"}
    #print("------------------------------------------------")
    #print("+    ",city.capitalize())
    hnev=ansicode(city).lower()

    prefix="https://ingatlan.com/szukites/"
    url=prefix+"elado+haz+"+hnev+"+"+str(pmin)+"-"+str(pmax)+"-mFt"
    #print(url)
    res=requests.get(url,headers=header)
    #print (res.text)
    soup=BeautifulSoup(res.text,"html.parser")
    #print(soup)
    cardcontainer=soup.find_all("a", class_="listing__thumbnail js-listing-active-area")
    #print(cardcontainer)
    urllista=[]
    pricelist=[]
    for cc in cardcontainer:
        #print(cc['href'])
        urlf=cc["href"]
        urllist=urlf.split("/")
        code=urllist[-1]
        #print(code)
        urllista.append(code)

        #pricecontainer=cc.find_all("div",class_="figFav")
        #pricelist.append(pricecontainer[0]["data-price"])    
    
    
    return(urllista)
 
    '''
    for i in enumerate(urllist):
        link=prefix+i[1]
        o=(queryallinfo(link))
        name=o[0]
        num=o[1]
        haz=o[2]
        telek=o[3]
        print(f"{city:>15};{i[0]:>4}; {name:>40}; {num:>18}; {haz:>10}; {telek:>10}; {pricelist[i[0]]:>10}; Ft ;  {link:<1000}",file=openfile)
        print(f"{city:>15};{i[0]:>4}; {name:>40}; {num:>18}; {haz:>10}; {telek:>10}; {pricelist[i[0]]:>10}; Ft ;  {link:<1000}")
        #
    print("----------------  END  ------------------------")
'''

        
        
def query_all_link(city,querymin=0,querymax=150,querydelta=5):
    allist=[]
    print(city+":",end="")
    prefix="https://ingatlan.com/"
    for i in range(querymin,querymax,querydelta):
        #print(i,i+querydelta)
        allist=allist+querycity(city,pmin=i,pmax=i+querydelta)
        print(".", end="")
    allist=list(set(allist))
    allist=[city,prefix]+allist
    #print("")
    #print(allist)
    return(allist)


Mounted at /content/drive


In [9]:
def query_all_info(url):
    header={'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36"}
   

    res=requests.get(url,headers=header)
    #print (res.text)
    soup=BeautifulSoup(res.text,"html.parser")
    allinfo=soup.find_all("span","parameter-value")
    
    meret=clearengunit(allinfo[0].text)
    telek=clearengunit(allinfo[1].text)
    szoba=clearengunit(allinfo[2].text)
    ar=onlynumber( allinfo[3].text)
    return([ar,szoba,meret,telek])
    
    


In [10]:
query_all_info("https://ingatlan.com/31195129")

['59,9', '5', '159', '320']

In [11]:

def printCSV(fname,arr,mode="a"):
    csvfile=open(fname,mode)
    for row in arr:
        for ind in row:
            print(ind,end=";",file=csvfile)
        print("",end="\n",file=csvfile)
    csvfile.close()




In [ ]:
#hely=open('/content/drive/My Drive/Ingatlan/fokusz.txt','r')
telepuleslista=open("/content/drive/My Drive/Ingatlan/helyisegek3.txt",encoding="utf-8-sig")
hely=[]
for telepules in telepuleslista:
    hely.append(telepules.strip())
telepuleslista.close()
print(hely)




#hely=["Szomor","Gyermely"]
print(hely)
prefix="https://ingatlan.com/"
all_link=[]
database=[]
exportfile="/content/drive/My Drive/Ingatlan/ingatlan_com_links.txt"


printCSV(exportfile,database,"a")    
for lak in hely:
    telepules=lak.strip()
    all_link=[]
    all_link.append(query_all_link(telepules,querymax=150))

    time.sleep(5)
    for link_lista in all_link:
        database=[]
        for data in link_lista[2:]:
            url=prefix+data
            print(url)
            homedata=query_all_info(url)
            database.append([link_lista[0],url,data]+homedata)
        printCSV(exportfile,database)

#print(database)  



['Gyepükaján', 'Gyermely', 'Gyód', 'Gyomaendrőd', 'Gyóró', 'Gyömöre', 'Gyömrő', 'Gyöngyfa', 'Gyöngyös', 'Gyöngyösfalu', 'Gyöngyöshalász', 'Gyöngyösmellék', 'Gyöngyösoroszi', 'Gyöngyöspata', 'Gyöngyössolymos', 'Gyöngyöstarján', 'Gyönk', 'Győr', 'Győrasszonyfa', 'Györe', 'Györgytarló', 'Györköny', 'Győrladamér', 'Győröcske', 'Győrság', 'Győrsövényház', 'Győrszemere', 'Győrtelek', 'Győrújbarát', 'Győrújfalu', 'Győrvár', 'Győrzámoly', 'Gyugy', 'Gyula', 'Gyulaháza', 'Gyulaj', 'Gyulakeszi', 'Gyúró', 'Gyügye', 'Gyüre', 'Gyűrűs', 'Hács', 'Hagyárosbörönd', 'Hahót', 'Hajdúbagos', 'Hajdúböszörmény', 'Hajdúdorog', 'Hajdúhadház', 'Hajdúnánás', 'Hajdúsámson', 'Hajdúszoboszló', 'Hajdúszovát', 'Hajmás', 'Hajmáskér', 'Hajós', 'Halastó', 'Halászi', 'Halásztelek', 'Halimba', 'Halmaj', 'Halmajugra', 'Halogy', 'Hangács', 'Hangony', 'Hantos', 'Harasztifalu', 'Harc', 'Harka', 'Harkakötöny', 'Harkány', 'Háromfa', 'Háromhuta', 'Harsány', 'Hárskút', 'Harta', 'Hásságy', 'Hatvan', 'Hédervár', 'Hedrehely', 'Hegyes

In [ ]:
knxvv